In [3]:
# -*- coding: utf-8 -*-
import pyodbc
from faker import Faker
import random
from datetime import datetime, timedelta, date

fake = Faker('es_ES')

# ---------- CONEXIÓN ----------
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=localhost\SQLEXPRESS;'
    'DATABASE=rrhh_database;'
    'UID=bestevez;'
    'PWD=Lianli3915-'
)
cursor = conn.cursor()
cursor.fast_executemany = True          # imprescindible para inserts masivos

# ---------- CONFIGURACIÓN ----------
NUM_TIENDAS       = 5
NUM_PUESTOS       = 8
NUM_EMPLEADOS     = 1000
FICHAJES_BATCH    = 10_000              # commit cada N inserts en Fichajes
START_FICH        = date(2024, 1, 1)
END_FICH          = date(2025, 12, 31)

# ---------- 1. TABLAS MAESTRAS ----------
def insert_tiendas():
    for i in range(1, NUM_TIENDAS + 1):
        cursor.execute("""
            INSERT INTO Tiendas
            (ID_Tienda, Nombre_Tienda, Ciudad, Provincia, Pais, Fecha_Apertura, Metros_Cuadrados)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (
            i,
            f"Tienda {i} {fake.city_suffix()}",
            fake.city(),
            fake.state(),
            "España",
            fake.date_between(start_date='-4000d', end_date='-365d'),
            random.randint(400, 1500)
        ))

def insert_puestos():
    puestos_demo = [
        ("Dependiente/a", "Ventas", "Operativo"),
        ("Encargado/a", "Ventas", "Mando intermedio"),
        ("Cajero/a", "Operaciones", "Operativo"),
        ("Visual merchandiser", "Marketing", "Especialista"),
        ("Responsable RR. HH.", "RRHH", "Mando medio"),
        ("Almacén", "Logística", "Operativo"),
        ("Gerente tienda", "Dirección tienda", "Manager"),
        ("Personal limpieza", "Servicios", "Operativo"),
    ]
    for i, (nombre, dept, nivel) in enumerate(puestos_demo, start=1):
        cursor.execute("""
            INSERT INTO Puestos (ID_Puesto, Nombre_Puesto, Departamento, Nivel)
            VALUES (?, ?, ?, ?)
        """, (i, nombre, dept, nivel))

# ---------- 2. EMPLEADOS ----------
def insert_empleados():
    for i in range(1, NUM_EMPLEADOS + 1):
        puesto_id = random.randint(1, NUM_PUESTOS)
        tienda_id = random.randint(1, NUM_TIENDAS)
        fecha_alta = fake.date_between(start_date='-2500d', end_date='-30d')
        jornada = random.choices(["Completa", "Parcial"], weights=[0.8, 0.2])[0]
        contrato = random.choices(
            ["Indefinido", "Temporal", "Prácticas"],
            weights=[0.65, 0.3, 0.05]
        )[0]
        salario = random.randint(18_000, 32_000)
        cursor.execute("""
            INSERT INTO Empleados
            (ID_Empleado, Nombre, Apellido, Genero, Fecha_Nacimiento,
             Fecha_Alta, Estado, ID_Puesto, ID_Tienda, Tipo_Contrato,
             Jornada, Salario_Base)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            i,
            fake.first_name(),
            fake.last_name(),
            random.choice(['M', 'F', 'O']),
            fake.date_between(start_date='-60y', end_date='-18y'),
            fecha_alta,
            "Activo",
            puesto_id,
            tienda_id,
            contrato,
            jornada,
            salario
        ))

# ---------- 3. FICHAJES DIARIOS ----------
def daterange(d1: date, d2: date):
    for n in range((d2 - d1).days + 1):
        yield d1 + timedelta(days=n)

def insert_fichajes():
    fichaje_id = 1
    pending = 0
    for emp_id in range(1, NUM_EMPLEADOS + 1):
        # mismo horario aproximado todo el año, con algo de ruido
        base_start = fake.random_int(min=7, max=11)   # 7-11h
        for day in daterange(START_FICH, END_FICH):
            # Salta fines de semana un 50 % (tiendas suelen abrir sábados)
            if day.weekday() >= 5 and random.random() < 0.5:
                continue
            # Entrada
            entrada = datetime.combine(day, datetime.min.time()) \
                      + timedelta(hours=base_start, minutes=random.randint(-15, 15))
            cursor.execute("""
                INSERT INTO Fichajes (ID_Fichaje, ID_Empleado, Fecha_Hora, Tipo, Dispositivo)
                VALUES (?, ?, ?, ?, ?)
            """, (fichaje_id, emp_id, entrada, "Entrada", "terminal-01"))
            fichaje_id += 1
            # Salida (8 h después aprox.)
            salida = entrada + timedelta(hours=8, minutes=random.randint(-10, 20))
            cursor.execute("""
                INSERT INTO Fichajes (ID_Fichaje, ID_Empleado, Fecha_Hora, Tipo, Dispositivo)
                VALUES (?, ?, ?, ?, ?)
            """, (fichaje_id, emp_id, salida, "Salida", "terminal-01"))
            fichaje_id += 1

            pending += 2
            if pending >= FICHAJES_BATCH:
                conn.commit()
                pending = 0
    if pending:
        conn.commit()

# ---------- 4. DATOS COMPLEMENTARIOS (muestras) ----------
def insert_ausencias(num_reg=800):
    for i in range(1, num_reg + 1):
        emp_id = random.randint(1, NUM_EMPLEADOS)
        inicio = fake.date_between(start_date='-365d', end_date='today')
        dur = random.randint(1, 5)
        fin = inicio + timedelta(days=dur)
        tipo = random.choices(
            ["Enfermedad", "Vacaciones", "Permiso", "No justificada"],
            weights=[0.5, 0.3, 0.15, 0.05]
        )[0]
        cursor.execute("""
            INSERT INTO Ausencias
            (ID_Ausencia, ID_Empleado, Fecha_Inicio, Fecha_Fin, Tipo_Ausencia, Comentario)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (i, emp_id, inicio, fin, tipo, fake.sentence(nb_words=6)))

def insert_movimientos(num_reg=120):
    motivos = ["Alta", "Baja", "Cambio_Puesto", "Traslado_Tienda"]
    for i in range(1, num_reg + 1):
        emp_id = random.randint(1, NUM_EMPLEADOS)
        fecha = fake.date_between(start_date='-730d', end_date='today')
        tipo = random.choice(motivos)
        detalle = fake.sentence(nb_words=8)
        cursor.execute("""
            INSERT INTO Movimientos_Laborales
            (ID_Movimiento, ID_Empleado, Fecha_Mov, Tipo_Mov, Detalle)
            VALUES (?, ?, ?, ?, ?)
        """, (i, emp_id, fecha, tipo, detalle))

def insert_evaluaciones(num_reg=1500):
    for i in range(1, num_reg + 1):
        emp_id = random.randint(1, NUM_EMPLEADOS)
        fecha = fake.date_between(start_date='-730d', end_date='today')
        puntuacion = round(random.uniform(2.5, 5.0), 2)
        cursor.execute("""
            INSERT INTO Evaluaciones
            (ID_Evaluacion, ID_Empleado, Fecha_Eval, Puntuacion, Comentarios, Evaluador)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (i, emp_id, fecha, puntuacion, fake.sentence(5), fake.name()))

def insert_formaciones(num_reg=900):
    cursos = ["Moda Sostenible", "Atención al cliente", "Visual Merchandising",
              "Gestión de Inventario", "Liderazgo de equipos", "Prevención Riesgos"]
    for i in range(1, num_reg + 1):
        emp_id = random.randint(1, NUM_EMPLEADOS)
        curso = random.choice(cursos)
        inicio = fake.date_between(start_date='-730d', end_date='-10d')
        fin = inicio + timedelta(days=random.choice([1, 2, 3, 5]))
        horas = random.randint(4, 24)
        cursor.execute("""
            INSERT INTO Formaciones
            (ID_Formacion, ID_Empleado, Curso, Fecha_Inicio, Fecha_Fin, Horas, Certificado)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (i, emp_id, curso, inicio, fin, horas, f"Certificado {curso}"))

def insert_costes_salariales():
    coste_id = 1
    for emp_id in range(1, NUM_EMPLEADOS + 1):
        for mes in range(0, 18):                      # 18 meses retrospectivos
            ref_mes = datetime.today().replace(day=1) - timedelta(days=30*mes)
            cursor.execute("""
                INSERT INTO Costes_Salariales
                (ID_Coste, ID_Empleado, Mes, Salario_Fijo, Variables, Seguridad_Social)
                VALUES (?, ?, ?, ?, ?, ?)
            """, (
                coste_id,
                emp_id,
                ref_mes.date(),
                random.randint(1200, 2200),
                random.randint(0, 400),
                random.randint(400, 650)
            ))
            coste_id += 1

# ---------- EJECUCIÓN ----------
print("Insertando tiendas y puestos…")
insert_tiendas()
insert_puestos()
print("Insertando empleados…")
insert_empleados()
conn.commit()

print("Generando fichajes (puede tardar)…")
insert_fichajes()

print("Insertando ausencias, movimientos, evaluaciones, formaciones y costes…")
insert_ausencias()
insert_movimientos()
insert_evaluaciones()
insert_formaciones()
insert_costes_salariales()

conn.commit()
cursor.close()
conn.close()
print("¡Datos de People Analytics generados correctamente!")


Insertando tiendas y puestos…
Insertando empleados…
Generando fichajes (puede tardar)…
Insertando ausencias, movimientos, evaluaciones, formaciones y costes…
¡Datos de People Analytics generados correctamente!
